In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cvxpy as cp
import shutil
import sys
import os.path
from pyomo.environ import *

In [ ]:
model = ConcreteModel()

# declare decision variables
model.x = Var(domain=NonNegativeReals)

# declare objective
model.profit = Objective(
    expr = 40*model.x,
    sense = maximize)

# declare constraints
model.demand = Constraint(expr = model.x <= 40)
model.laborA = Constraint(expr = model.x <= 80)
model.laborB = Constraint(expr = 2*model.x <= 100)

# solve
SolverFactory('cplex_direct').solve(model).write()

In [ ]:
print(f"Profit = {model.profit()} per week")
print(f"X = {model.x()} units per week")

In [ ]:
model = ConcreteModel()

# declare decision variables
model.y = Var(domain=NonNegativeReals)

# declare objective
model.profit = Objective(
    expr = 30*model.y,
    sense = maximize)

# declare constraints
model.laborA = Constraint(expr = model.y <= 80)
model.laborB = Constraint(expr = model.y <= 100)

# solve
SolverFactory('cplex_direct').solve(model).write()

In [ ]:
model = ConcreteModel()

# declare decision variables
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# declare objective
model.profit = Objective(
    expr = 40*model.x + 30*model.y,
    sense = maximize)

# declare constraints
model.demand = Constraint(expr = model.x <= 40)
model.laborA = Constraint(expr = model.x + model.y <= 80)
model.laborB = Constraint(expr = 2*model.x + model.y <= 100)

# solve
SolverFactory('cplex_direct').solve(model).write()

In [ ]:
# display solution
print(f"Profit = {model.profit()}")
print(f"Units of X = {model.x()}")
print(f"Units of Y = {model.y()}")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.set_aspect('equal')
ax.axis([0, 100, 0, 100])
ax.set_xlabel('X Production')
ax.set_ylabel('Y Production')

# Labor A constraint
x = np.array([0, 80])
ax.plot(x, 80 - x, 'r', lw=2)

# Labor B constraint
x = np.array([0, 50])
ax.plot(x, 100 - 2*x, 'b', lw=2)

# Demand constraint
ax.plot([40, 40], [0, 100], 'g', lw=2)

ax.legend(['Labor A Constraint', 'Labor B Constraint', 'Demand Constraint'])
ax.fill_between([0, 80, 100], [80, 0,0 ], [100, 100, 100], color='r', alpha=0.15)
ax.fill_between([0, 50, 100], [100, 0, 0], [100, 100, 100], color='b', alpha=0.15)
ax.fill_between([40, 100], [0, 0], [100, 100], color='g', alpha=0.15)

# Contours of constant profit
x = np.array([0, 100])
for p in np.linspace(0, 3600, 10):
    y = (p - 40*x)/30
    ax.plot(x, y, 'y--')
    
arrowprops = dict(shrink=.1, width=1, headwidth=5)

# Optimum
ax.plot(20, 60, 'r.', ms=20)
ax.annotate('Mixed Product Strategy', xy=(20, 60), xytext=(50, 70), arrowprops=arrowprops)

ax.plot(0, 80, 'b.', ms=20)
ax.annotate('Y Only', xy=(0, 80), xytext=(20, 90), arrowprops=arrowprops)

ax.plot(40, 0, 'b.', ms=20)
ax.annotate('X Only', xy=(40, 0), xytext=(70, 20), arrowprops=arrowprops)

ax.text(4, 23, 'Increasing Profit')
ax.annotate('', xy=(20, 15), xytext=(0,0), arrowprops=arrowprops)

fname = 'LPprog01.png'
fname = os.path.join('figures', fname) if os.path.exists('figures') else fname
plt.savefig(fname, bbox_inches='tight')

In [ ]:
model = ConcreteModel()

# declare decision variables
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# declare objective
model.profit = Objective(
    expr = 40*model.x + 30*model.y,
    sense = maximize)

# declare constraints
model.demand = Constraint(expr = model.x <= 40)
model.laborA = Constraint(expr = model.x + model.y <= 80)
model.laborB = Constraint(expr = 2*model.x + model.y <= 100)

# solve
SolverFactory('cplex_direct').solve(model)

print(f"P = {model.profit()}")
print(f"x = {model.x()}")
print(f"y = {model.y()}")

In [ ]:
model = ConcreteModel()

# for access to dual solution for constraints
model.dual = Suffix(direction=Suffix.IMPORT)

# declare decision variables
model.x = Var(domain=NonNegativeReals)
model.y = Var(domain=NonNegativeReals)

# declare objective
model.profit = Objective(
    expr = 40*model.x + 30*model.y,
    sense = maximize)

# declare constraints
model.demand = Constraint(expr = model.x <= 40)
model.laborA = Constraint(expr = model.x + model.y <= 80)
model.laborB = Constraint(expr = 2*model.x + model.y <= 100)

# solve
SolverFactory('cplex_direct').solve(model)

print("\nSolution")
print(f"x = {model.x()}")
print(f"y = {model.y()}")

print("\nSensitivity Analysis")
print(f"y_demand = {-model.dual[model.demand]}")
print(f"y_laborA = {-model.dual[model.laborA]}")
print(f"y_laborB = {-model.dual[model.laborB]}")

In [ ]:
eta_plus=0.96 # charging efficiency
eta_minus=0.96 # discharging efficiency
Emax=450 # SOC upper limit
Emin=100 # SOC lower limit
E_init=250 # initial state of charge
P_B_plus_max=100 # charging power limit
P_B_minus_max=100 # discharging power limit

opt_load=load #declaring optimal load
n=96 #declaring number of timestpes for each optimization
del_t=1/4 #time delta
d=length(load)/n #number of days

# tou energy charge array
lg = [31, 16, 25, 20, 4]
pk = ['off', 'mid', 'on', 'mid', 'off']
alpha = np.array([])
for i in range(len(lg)):
    if pk[i] == 'on':
        mult = 0.1079
    elif pk[i] == 'mid':
        mult = 0.0874
    elif pk[i] == 'off':
        mult = 0.0755
    alpha = np.append(alpha, (mult * np.ones(lg[i])))
# OFF1=0.0755*ones(31,lg)
# MID1=0.0874*ones(16,1)
# ON=0.1079*ones(25,1)
# MID2=0.0874*ones(20,1)
# OFF2=0.0755*ones(4,1)
# alpha=[OFF1MID1ONMID2OFF2]

# tou demand charge matrix
beta_OFF_val=1.53
beta_MID_val=3.13
beta_ON_val=7.06

obj = cp.Minimize(alpha * P_G * del_t + cp.max(beta_OFF @ P_G) + cp.max(beta_MID @ P_G) + cp.max(beta_ON @ P_G))

for t in range(1,n):
    cons_temp = [
        E_B[t] >= Emin,
        E_B[t] <= Emax,
        P_B_plus[t] >= 0,
        P_B_plus[t] <= P_B_plus_max,
        P_B_minus[t] >= 0,
        P_B_minus[t] <= P_B_minus_max,
        P_SL[t] + P_B_plus[t]/eta_plus == P_S[t],
        P_SL[t] + P_G(t) + P_B_minus[t]*eta_minus == P_L[t],
        P_SL[t] >= 0
    ]
    cons += cons_temp
prob = cp.Problem(obj, cons)

print(prob.solve(solver=cp.SCS))
print(prob.status)

In [ ]:
import cvxpy as cp
import numpy as np

# Problem data.
m = 30
n = 20
np.random.seed(1)
A = np.random.randn(m, n)
b = np.random.randn(m)

# Construct the problem.
x = cp.Variable(n)
objective = cp.Minimize(cp.sum_squares(A @ x - b))
constraints = [0 <= x, x <= 1]
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print(x.value)
# The optimal Lagrange multiplier for a constraint is stored in
# `constraint.dual_value`.
print(constraints[0].dual_value)

In [ ]:
import cvxpy as cp
n = 100
init = 10
A = cp.Variable(n)
B = cp.Variable(n)
C = cp.Variable(n)
X = cp.Variable(n)

obj = cp.Minimize(cp.sum(A) + cp.max(B))
# TODO automate introduction of variables.
cons = [
   X[0] == init,
   A[0] >= 1,
   B[0] <= 1,
   C[0] >= 1
]

for t in range(1,n):
    cons2 = [
        X[t] == X[t - 1] + A[t - 1] + B[t - 1] + C[t - 1],
        A[t] >= 1,
        B[t] <= 1,
        C[t] >= 1
    ]
    cons += cons2
prob = cp.Problem(obj, cons)

print(prob.solve())
print(prob.status)

In [ ]:
lg = [31, 16, 25, 20, 4]
pk = ['off', 'mid', 'on', 'mid', 'off']
alpha = np.array([])
for i in range(len(lg)):
    if pk[i] == 'on':
        mult = 0.1079
    elif pk[i] == 'mid':
        mult = 0.0874
    elif pk[i] == 'off':
        mult = 0.0755
    alpha = np.append(alpha, (mult * np.ones(lg[i])))
alpha

In [ ]:
lg = [31, 16, 25, 20, 4]
pk = ['off', 'mid', 'on', 'mid', 'off']
val = [[0.1709, 0, 0], [0, 0.0874, 0], [0, 0, 0.0755]]
beta = {}
for i in range(len(val)):
    beta_i = np.array([])
    for j in range(len(lg)):
        if pk[j] == 'on':
            mult = val[0][i]
        elif pk[j] == 'mid':
            mult = val[1][i]
        elif pk[j] == 'off':
            mult = val[2][i]
        beta_i = np.append(beta_i, (mult * np.ones(lg[j])))
    beta[i] = beta_i
beta_ON = np.zeros((96, 96))
np.fill_diagonal(beta_ON, beta[0])
beta_MID = np.zeros((96, 96))
np.fill_diagonal(beta_MID, beta[1])
beta_OFF = np.zeros((96, 96))
np.fill_diagonal(beta_OFF, beta[2])

In [ ]:
df = pd.read_csv('/home/sigi_laptop/Documents/Research/NAPS_2023/Code/optimization_1084_2022_2023.csv')
df

In [ ]:
df.solar.plot()

In [ ]:
load = df.building_load.to_numpy()
solar = df.solar.to_numpy()
eta_plus=0.96 # charging efficiency
eta_minus=0.96 # discharging efficiency
Emax=450 # SOC upper limit
Emin=100 # SOC lower limit
E_init=250 # initial state of charge
P_B_plus_max=100 # charging power limit
P_B_minus_max=100 # discharging power limit

opt_load=load #declaring optimal load
n=96 #declaring number of timestpes for each optimization
del_t=1/4 #time delta
d=int(len(load) / n )#number of days


# tou demand charge matrix
beta_OFF_val=1.53
beta_MID_val=3.13
beta_ON_val=7.06
p = np.array([])
for i in range(d):
    cons = []
    P_S = solar[int(n*i) : int(n*i + n)]
    P_L = load[int(n*i) : int(n*i + n)]

    P_G = cp.Variable(n)
    E_B = cp.Variable(n)
    P_B_plus = cp.Variable(n)
    P_B_minus = cp.Variable(n)
    P_SL = cp.Variable(n)
 
    obj = cp.Minimize(alpha @ P_G * del_t + cp.max(beta_OFF @ P_G) + cp.max(beta_MID @ P_G) + cp.max(beta_ON @ P_G))
    for t in range(n):
        if t == 0:
           cons_temp = [
                E_B[t] == E_init,
                E_B[t] >= Emin,
                E_B[t] <= Emax,
                P_B_plus[t] >= 0,
                P_B_plus[t] <= P_B_plus_max,
                P_B_minus[t] >= 0,
                P_B_minus[t] <= P_B_minus_max,
                P_SL[t] + P_B_plus[t]/eta_plus == P_S[t],
                P_SL[t] + P_G[t] + P_B_minus[t]*eta_minus == P_L[t],
                P_SL[t] >= 0
            ]
        else:
            cons_temp = [
                E_B[t] == E_B[t - 1] + del_t*(P_B_plus[t - 1] - P_B_minus[t - 1]),
                E_B[t] >= Emin,
                E_B[t] <= Emax,
                P_B_plus[t] >= 0,
                P_B_plus[t] <= P_B_plus_max,
                P_B_minus[t] >= 0,
                P_B_minus[t] <= P_B_minus_max,
                P_SL[t] + P_B_plus[t]/eta_plus == P_S[t],
                P_SL[t] + P_G[t] + P_B_minus[t]*eta_minus == P_L[t],
                P_SL[t] >= 0
            ]
        cons += cons_temp
    prob = cp.Problem(obj, cons)
    p = np.append(p, P_G)
    E_init  = E_B[n - 1]
print(prob.solve(solver=cp.SCS, verbose = True))
print(prob.status)

In [ ]:
prob.P_G

In [ ]:
test = pd.DataFrame(P_L)

In [ ]:
test[0].plot()